In [20]:
!pip install sentence_transformers

In [2]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("BAAI/bge-large-zh-v1.5")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\86183\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\86183\.cache\huggingface\hub\models--BAAI--bge-large-zh-v1.5. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/30.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/394 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/439k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [3]:
sentence = ["我是一名华中科技大学的学生",
            "我是一名华中科技大学的研究生",
            "我是一名华中科技大学的博士生",
            "我是一名华中科技大学的教授",
            "我是一名华中科技大学的博士后"]

embeddings = model.encode(sentence)
print(embeddings.shape)

(5, 1024)


In [23]:
similarities = model.similarity(embeddings, embeddings)
print(similarities)

tensor([[1.0000, 0.9856, 0.9205, 0.9651, 0.8813],
        [0.9856, 1.0000, 0.9199, 0.9606, 0.8704],
        [0.9205, 0.9199, 1.0000, 0.8827, 0.9715],
        [0.9651, 0.9606, 0.8827, 1.0000, 0.8923],
        [0.8813, 0.8704, 0.9715, 0.8923, 1.0000]])


可以看到已经打印了五个embedding的相似矩阵。

Milvus中的SentenceTransformerEmbeddingFunction类可以直接使用sentence-transformers库中的模型，将文本转换为向量。

In [24]:
#实例化SentenceTransformerEmbeddingFunction类
from pymilvus import model
sentence_transformer_ef = model.dense.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2", device="cpu")
#model_name指明模型名称，https://www.sbert.net/docs/sentence_transformer/pretrained_models.html
#device指明设备，cpu或gpu，如果你有GPU，指明cuda:n

生成文档的嵌入，用encode_documents, 查询词的嵌入，用encode_queries。

In [25]:
docs = ["我是一名华中科技大学的本科生",
        "我是一名华中科技大学的研究生",
        "我是一名华中科技大学的博士生",
        "我是一名华中科技大学的教授",
        "我是一名华中科技大学的博士后"]

doc_embeddings = sentence_transformer_ef.encode_documents(docs)

In [26]:
print("Embeddings:", doc_embeddings)

Embeddings: [array([ 2.48564258e-02,  1.23968266e-01,  9.68176723e-02, -1.01170801e-02,
        2.28363592e-02,  7.82503411e-02,  9.00707096e-02, -4.38400544e-03,
        7.21786246e-02,  3.64181660e-02,  1.23361111e-01, -1.11645311e-01,
       -5.17042950e-02, -5.32109067e-02,  3.39755509e-03,  1.06492676e-02,
       -4.39407043e-02,  1.46419425e-02, -8.81630257e-02,  3.11792754e-02,
        4.44559194e-02,  9.46250465e-03,  5.15955016e-02,  6.22264519e-02,
       -7.17773810e-02,  3.84809673e-02, -1.26256386e-03,  1.80335324e-02,
        4.51001488e-02, -1.20627861e-02, -3.34854424e-02,  2.25895923e-02,
        7.76904449e-02,  4.79382277e-02,  4.62265648e-02,  3.70206833e-02,
       -3.22460979e-02, -2.27387268e-02, -7.74688879e-03,  1.79591379e-03,
        1.35321599e-02, -7.45187560e-03,  7.45004192e-02, -5.17666824e-02,
        4.64094132e-02, -1.83333978e-02, -2.30367798e-02,  4.54552518e-03,
        4.34603319e-02,  7.17461854e-03, -1.17212601e-01, -3.00169196e-02,
       -5.13

In [27]:
print("dim:", sentence_transformer_ef.dim)

dim: 384


In [28]:
print("shape:",doc_embeddings[0].shape)

shape: (384,)


In [29]:
query = ["我是哪里的学生",
         "我的学位是什么"]
query_embeddings = sentence_transformer_ef.encode_queries(query)

In [30]:
print("Embeddings:", query_embeddings)

Embeddings: [array([-3.22065083e-04,  1.27307773e-01,  8.14300850e-02,  8.03022273e-03,
        1.32957483e-02,  4.28011687e-03,  7.76733235e-02, -3.58693935e-02,
        4.83983457e-02, -1.76279154e-02,  1.41495034e-01,  1.25088810e-03,
       -9.70786996e-03, -2.68726908e-02,  1.70472786e-02,  8.00468493e-03,
       -4.15833294e-02,  6.05199905e-03, -2.47620624e-02,  2.70401575e-02,
        2.32236329e-02,  4.47971420e-03,  3.95807102e-02,  1.04084827e-01,
       -1.72892492e-02, -4.97422032e-02, -8.78533628e-03, -9.28801950e-03,
        2.97963489e-02,  1.66751549e-03, -1.04208469e-01,  4.42610607e-02,
        2.97519453e-02,  2.19125282e-02,  3.48354988e-02,  4.85506393e-02,
       -2.83423811e-04, -2.84119342e-02, -4.85865511e-02,  2.02666484e-02,
       -3.48671935e-02, -4.01220471e-03,  6.06179051e-02, -1.94542576e-02,
        2.67322678e-02, -3.98502164e-02, -4.77369539e-02, -1.96589679e-02,
        5.40925823e-02, -2.49153264e-02, -1.17721796e-01,  4.78899200e-03,
       -3.71

In [31]:
print("dim:", sentence_transformer_ef.dim)

dim: 384


In [32]:
print("shape:",query_embeddings[0].shape)

shape: (384,)
